In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import tensorflow_probability as tfp

2024-01-23 09:13:09.574633: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [18]:
def get_train_and_test_splits(train_size, batch_size=1):
    # We prefetch with a buffer the same size as the dataset because the dataset
    # is very small and fits into memory.
    dataset = (
        tfds.load(name="wine_quality", as_supervised=True, split="train")
        .map(lambda x, y: (x, tf.cast(y, tf.float32)))
        .prefetch(buffer_size=dataset_size)
        .cache()
    )
    # We shuffle with a buffer the same size as the dataset.
    train_dataset = (
        dataset.take(train_size).shuffle(buffer_size=train_size).batch(batch_size)
    )
    test_dataset = dataset.skip(train_size).batch(batch_size)
    print(dataset)
    return train_dataset, test_dataset

In [3]:
hidden_units = [8, 8]
learning_rate = 0.001


def run_experiment(model, loss, train_dataset, test_dataset):

    model.compile(
        optimizer=keras.optimizers.RMSprop(learning_rate=learning_rate),
        loss=loss,
        metrics=[keras.metrics.RootMeanSquaredError()],
    )

    print("Start training the model...")
    model.fit(train_dataset, epochs=num_epochs, validation_data=test_dataset)
    print("Model training finished.")
    _, rmse = model.evaluate(train_dataset, verbose=0)
    print(f"Train RMSE: {round(rmse, 3)}")

    print("Evaluating model performance...")
    _, rmse = model.evaluate(test_dataset, verbose=0)
    print(f"Test RMSE: {round(rmse, 3)}")

In [4]:
FEATURE_NAMES = [
    "fixed acidity",
    "volatile acidity",
    "citric acid",
    "residual sugar",
    "chlorides",
    "free sulfur dioxide",
    "total sulfur dioxide",
    "density",
    "pH",
    "sulphates",
    "alcohol",
]


def create_model_inputs():
    inputs = {}
    for feature_name in FEATURE_NAMES:
        inputs[feature_name] = layers.Input(
            name=feature_name, shape=(1,), dtype=tf.float32
        )
    return inputs

In [5]:
def create_baseline_model():
    inputs = create_model_inputs()
    input_values = [value for _, value in sorted(inputs.items())]
    features = keras.layers.concatenate(input_values)
    features = layers.BatchNormalization()(features)

    # Create hidden layers with deterministic weights using the Dense layer.
    for units in hidden_units:
        features = layers.Dense(units, activation="sigmoid")(features)
    # The output is deterministic: a single point estimate.
    outputs = layers.Dense(units=1)(features)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [19]:
dataset_size = 4898
batch_size = 256
train_size = int(dataset_size * 0.85)
train_dataset, test_dataset = get_train_and_test_splits(train_size, batch_size)
train_dataset

<CacheDataset element_spec=({'alcohol': TensorSpec(shape=(), dtype=tf.float32, name=None), 'chlorides': TensorSpec(shape=(), dtype=tf.float32, name=None), 'citric acid': TensorSpec(shape=(), dtype=tf.float32, name=None), 'density': TensorSpec(shape=(), dtype=tf.float32, name=None), 'fixed acidity': TensorSpec(shape=(), dtype=tf.float32, name=None), 'free sulfur dioxide': TensorSpec(shape=(), dtype=tf.float32, name=None), 'pH': TensorSpec(shape=(), dtype=tf.float32, name=None), 'residual sugar': TensorSpec(shape=(), dtype=tf.float32, name=None), 'sulphates': TensorSpec(shape=(), dtype=tf.float64, name=None), 'total sulfur dioxide': TensorSpec(shape=(), dtype=tf.float32, name=None), 'volatile acidity': TensorSpec(shape=(), dtype=tf.float32, name=None)}, TensorSpec(shape=(), dtype=tf.float32, name=None))>


<_BatchDataset element_spec=({'alcohol': TensorSpec(shape=(None,), dtype=tf.float32, name=None), 'chlorides': TensorSpec(shape=(None,), dtype=tf.float32, name=None), 'citric acid': TensorSpec(shape=(None,), dtype=tf.float32, name=None), 'density': TensorSpec(shape=(None,), dtype=tf.float32, name=None), 'fixed acidity': TensorSpec(shape=(None,), dtype=tf.float32, name=None), 'free sulfur dioxide': TensorSpec(shape=(None,), dtype=tf.float32, name=None), 'pH': TensorSpec(shape=(None,), dtype=tf.float32, name=None), 'residual sugar': TensorSpec(shape=(None,), dtype=tf.float32, name=None), 'sulphates': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'total sulfur dioxide': TensorSpec(shape=(None,), dtype=tf.float32, name=None), 'volatile acidity': TensorSpec(shape=(None,), dtype=tf.float32, name=None)}, TensorSpec(shape=(None,), dtype=tf.float32, name=None))>

In [7]:
num_epochs = 100
mse_loss = keras.losses.MeanSquaredError()
baseline_model = create_baseline_model()
run_experiment(baseline_model, mse_loss, train_dataset, test_dataset)

Start training the model...
Epoch 1/100
 1/17 [>.............................] - ETA: 23s - loss: 43.0381 - root_mean_squared_error: 6.5603

2024-01-23 09:21:28.845976: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2024-01-23 09:21:28.849309: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


17/17 [==============================] - 2s 38ms/step - loss: 41.9571 - root_mean_squared_error: 6.4774 - val_loss: 39.7342 - val_root_mean_squared_error: 6.3035
Epoch 2/100
17/17 [==============================] - 0s 5ms/step - loss: 39.7999 - root_mean_squared_error: 6.3087 - val_loss: 37.9243 - val_root_mean_squared_error: 6.1583
Epoch 3/100
17/17 [==============================] - 0s 6ms/step - loss: 37.9769 - root_mean_squared_error: 6.1625 - val_loss: 36.1840 - val_root_mean_squared_error: 6.0153
Epoch 4/100
17/17 [==============================] - 0s 6ms/step - loss: 36.2213 - root_mean_squared_error: 6.0184 - val_loss: 34.5289 - val_root_mean_squared_error: 5.8761
Epoch 5/100
17/17 [==============================] - 0s 6ms/step - loss: 34.4954 - root_mean_squared_error: 5.8733 - val_loss: 32.9482 - val_root_mean_squared_error: 5.7400
Epoch 6/100
17/17 [==============================] - 0s 5ms/step - loss: 32.7973 - root_mean_squared_error: 5.7269 - val_loss: 31.3932 - val_root_

17/17 [==============================] - 0s 4ms/step - loss: 1.0458 - root_mean_squared_error: 1.0227 - val_loss: 0.9791 - val_root_mean_squared_error: 0.9895
Epoch 49/100
17/17 [==============================] - 0s 4ms/step - loss: 0.9521 - root_mean_squared_error: 0.9757 - val_loss: 0.9060 - val_root_mean_squared_error: 0.9519
Epoch 50/100
17/17 [==============================] - 0s 4ms/step - loss: 0.8813 - root_mean_squared_error: 0.9388 - val_loss: 0.8527 - val_root_mean_squared_error: 0.9234
Epoch 51/100
17/17 [==============================] - 0s 4ms/step - loss: 0.8304 - root_mean_squared_error: 0.9113 - val_loss: 0.8220 - val_root_mean_squared_error: 0.9067
Epoch 52/100
17/17 [==============================] - 0s 4ms/step - loss: 0.8004 - root_mean_squared_error: 0.8947 - val_loss: 0.8050 - val_root_mean_squared_error: 0.8972
Epoch 53/100
17/17 [==============================] - 0s 4ms/step - loss: 0.7851 - root_mean_squared_error: 0.8861 - val_loss: 0.8022 - val_root_mean_squ

Epoch 96/100
17/17 [==============================] - 0s 5ms/step - loss: 0.7205 - root_mean_squared_error: 0.8488 - val_loss: 0.7464 - val_root_mean_squared_error: 0.8640
Epoch 97/100
17/17 [==============================] - 0s 4ms/step - loss: 0.7206 - root_mean_squared_error: 0.8489 - val_loss: 0.7446 - val_root_mean_squared_error: 0.8629
Epoch 98/100
17/17 [==============================] - 0s 4ms/step - loss: 0.7169 - root_mean_squared_error: 0.8467 - val_loss: 0.7422 - val_root_mean_squared_error: 0.8615
Epoch 99/100
17/17 [==============================] - 0s 4ms/step - loss: 0.7156 - root_mean_squared_error: 0.8459 - val_loss: 0.7396 - val_root_mean_squared_error: 0.8600
Epoch 100/100
17/17 [==============================] - 0s 4ms/step - loss: 0.7132 - root_mean_squared_error: 0.8445 - val_loss: 0.7413 - val_root_mean_squared_error: 0.8610
Model training finished.
Train RMSE: 0.844
Evaluating model performance...
Test RMSE: 0.861


In [8]:
sample = 10
examples, targets = list(test_dataset.unbatch().shuffle(batch_size * 10).batch(sample))[
    0
]

predicted = baseline_model(examples).numpy()
for idx in range(sample):
    print(f"Predicted: {round(float(predicted[idx][0]), 1)} - Actual: {targets[idx]}")

Predicted: 6.0 - Actual: 8.0
Predicted: 6.1 - Actual: 5.0
Predicted: 6.1 - Actual: 6.0
Predicted: 5.8 - Actual: 5.0
Predicted: 6.0 - Actual: 5.0
Predicted: 6.1 - Actual: 7.0
Predicted: 6.1 - Actual: 5.0
Predicted: 6.0 - Actual: 7.0
Predicted: 6.0 - Actual: 5.0
Predicted: 6.0 - Actual: 6.0


In [9]:
# Define the prior weight distribution as Normal of mean=0 and stddev=1.
# Note that, in this example, the we prior distribution is not trainable,
# as we fix its parameters.
def prior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    prior_model = keras.Sequential(
        [
            tfp.layers.DistributionLambda(
                lambda t: tfp.distributions.MultivariateNormalDiag(
                    loc=tf.zeros(n), scale_diag=tf.ones(n)
                )
            )
        ]
    )
    return prior_model


# Define variational posterior weight distribution as multivariate Gaussian.
# Note that the learnable parameters for this distribution are the means,
# variances, and covariances.
def posterior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    posterior_model = keras.Sequential(
        [
            tfp.layers.VariableLayer(
                tfp.layers.MultivariateNormalTriL.params_size(n), dtype=dtype
            ),
            tfp.layers.MultivariateNormalTriL(n),
        ]
    )
    return posterior_model

In [10]:
def create_bnn_model(train_size):
    inputs = create_model_inputs()
    features = keras.layers.concatenate(list(inputs.values()))
    features = layers.BatchNormalization()(features)

    # Create hidden layers with weight uncertainty using the DenseVariational layer.
    for units in hidden_units:
        features = tfp.layers.DenseVariational(
            units=units,
            make_prior_fn=prior,
            make_posterior_fn=posterior,
            kl_weight=1 / train_size,
            activation="sigmoid",
        )(features)

    # The output is deterministic: a single point estimate.
    outputs = layers.Dense(units=1)(features)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [11]:
num_epochs = 500
train_sample_size = int(train_size * 0.3)
small_train_dataset = train_dataset.unbatch().take(train_sample_size).batch(batch_size)

bnn_model_small = create_bnn_model(train_sample_size)
run_experiment(bnn_model_small, mse_loss, small_train_dataset, test_dataset)

Start training the model...
Epoch 1/500
5/5 [==============================] - 5s 349ms/step - loss: 53.0552 - root_mean_squared_error: 7.2831 - val_loss: 55.6611 - val_root_mean_squared_error: 7.4598
Epoch 2/500
5/5 [==============================] - 0s 13ms/step - loss: 52.7193 - root_mean_squared_error: 7.2598 - val_loss: 50.7723 - val_root_mean_squared_error: 7.1244
Epoch 3/500
5/5 [==============================] - 0s 14ms/step - loss: 48.6775 - root_mean_squared_error: 6.9759 - val_loss: 52.5980 - val_root_mean_squared_error: 7.2515
Epoch 4/500
5/5 [==============================] - 0s 14ms/step - loss: 49.3860 - root_mean_squared_error: 7.0268 - val_loss: 49.8171 - val_root_mean_squared_error: 7.0569
Epoch 5/500
5/5 [==============================] - 0s 14ms/step - loss: 49.5530 - root_mean_squared_error: 7.0383 - val_loss: 47.1204 - val_root_mean_squared_error: 6.8634
Epoch 6/500
5/5 [==============================] - 0s 14ms/step - loss: 48.5092 - root_mean_squared_error: 6.96

5/5 [==============================] - 0s 16ms/step - loss: 33.2177 - root_mean_squared_error: 5.7620 - val_loss: 31.5698 - val_root_mean_squared_error: 5.6172
Epoch 49/500
5/5 [==============================] - 0s 12ms/step - loss: 31.4421 - root_mean_squared_error: 5.6057 - val_loss: 30.7810 - val_root_mean_squared_error: 5.5463
Epoch 50/500
5/5 [==============================] - 0s 10ms/step - loss: 33.5474 - root_mean_squared_error: 5.7907 - val_loss: 32.5042 - val_root_mean_squared_error: 5.6999
Epoch 51/500
5/5 [==============================] - 0s 11ms/step - loss: 32.0089 - root_mean_squared_error: 5.6561 - val_loss: 29.6803 - val_root_mean_squared_error: 5.4464
Epoch 52/500
5/5 [==============================] - 0s 10ms/step - loss: 31.8161 - root_mean_squared_error: 5.6393 - val_loss: 29.1065 - val_root_mean_squared_error: 5.3931
Epoch 53/500
5/5 [==============================] - 0s 13ms/step - loss: 30.4792 - root_mean_squared_error: 5.5194 - val_loss: 29.7716 - val_root_me

5/5 [==============================] - 0s 13ms/step - loss: 18.8911 - root_mean_squared_error: 4.3435 - val_loss: 19.1798 - val_root_mean_squared_error: 4.3772
Epoch 96/500
5/5 [==============================] - 0s 13ms/step - loss: 18.5532 - root_mean_squared_error: 4.3049 - val_loss: 17.5598 - val_root_mean_squared_error: 4.1873
Epoch 97/500
5/5 [==============================] - 0s 13ms/step - loss: 18.6189 - root_mean_squared_error: 4.3127 - val_loss: 19.5673 - val_root_mean_squared_error: 4.4218
Epoch 98/500
5/5 [==============================] - 0s 13ms/step - loss: 19.0433 - root_mean_squared_error: 4.3613 - val_loss: 17.5290 - val_root_mean_squared_error: 4.1839
Epoch 99/500
5/5 [==============================] - 0s 16ms/step - loss: 18.2491 - root_mean_squared_error: 4.2694 - val_loss: 18.0756 - val_root_mean_squared_error: 4.2498
Epoch 100/500
5/5 [==============================] - 0s 16ms/step - loss: 17.1955 - root_mean_squared_error: 4.1436 - val_loss: 17.2982 - val_root_m

5/5 [==============================] - 0s 13ms/step - loss: 10.1416 - root_mean_squared_error: 3.1805 - val_loss: 8.4428 - val_root_mean_squared_error: 2.9004
Epoch 143/500
5/5 [==============================] - 0s 12ms/step - loss: 9.2337 - root_mean_squared_error: 3.0347 - val_loss: 9.1354 - val_root_mean_squared_error: 3.0178
Epoch 144/500
5/5 [==============================] - 0s 12ms/step - loss: 9.1237 - root_mean_squared_error: 3.0158 - val_loss: 8.8320 - val_root_mean_squared_error: 2.9672
Epoch 145/500
5/5 [==============================] - 0s 13ms/step - loss: 8.5726 - root_mean_squared_error: 2.9230 - val_loss: 8.5642 - val_root_mean_squared_error: 2.9212
Epoch 146/500
5/5 [==============================] - 0s 13ms/step - loss: 8.1873 - root_mean_squared_error: 2.8559 - val_loss: 8.2305 - val_root_mean_squared_error: 2.8642
Epoch 147/500
5/5 [==============================] - 0s 13ms/step - loss: 8.7333 - root_mean_squared_error: 2.9504 - val_loss: 7.6760 - val_root_mean_squ

Epoch 190/500
5/5 [==============================] - 0s 16ms/step - loss: 3.2585 - root_mean_squared_error: 1.7955 - val_loss: 3.0086 - val_root_mean_squared_error: 1.7240
Epoch 191/500
5/5 [==============================] - 0s 12ms/step - loss: 2.9803 - root_mean_squared_error: 1.7161 - val_loss: 2.9988 - val_root_mean_squared_error: 1.7226
Epoch 192/500
5/5 [==============================] - 0s 15ms/step - loss: 3.0229 - root_mean_squared_error: 1.7288 - val_loss: 2.7692 - val_root_mean_squared_error: 1.6547
Epoch 193/500
5/5 [==============================] - 0s 15ms/step - loss: 2.6489 - root_mean_squared_error: 1.6150 - val_loss: 2.7290 - val_root_mean_squared_error: 1.6416
Epoch 194/500
5/5 [==============================] - 0s 14ms/step - loss: 2.6310 - root_mean_squared_error: 1.6112 - val_loss: 2.7332 - val_root_mean_squared_error: 1.6436
Epoch 195/500
5/5 [==============================] - 0s 16ms/step - loss: 2.8046 - root_mean_squared_error: 1.6648 - val_loss: 2.3048 - val_

...

Epoch 285/500
5/5 [==============================] - 0s 12ms/step - loss: 0.7961 - root_mean_squared_error: 0.8761 - val_loss: 0.8312 - val_root_mean_squared_error: 0.8952
Epoch 286/500
5/5 [==============================] - 0s 12ms/step - loss: 0.7834 - root_mean_squared_error: 0.8699 - val_loss: 0.8322 - val_root_mean_squared_error: 0.8958
Epoch 287/500
5/5 [==============================] - 0s 13ms/step - loss: 0.7801 - root_mean_squared_error: 0.8664 - val_loss: 0.8025 - val_root_mean_squared_error: 0.8796
Epoch 288/500
5/5 [==============================] - 0s 12ms/step - loss: 0.7497 - root_mean_squared_error: 0.8491 - val_loss: 0.8047 - val_root_mean_squared_error: 0.8812
Epoch 289/500
5/5 [==============================] - 0s 12ms/step - loss: 0.7762 - root_mean_squared_error: 0.8654 - val_loss: 0.7996 - val_root_mean_squared_error: 0.8799
Epoch 290/500
5/5 [==============================] - 0s 11ms/step - loss: 0.7749 - root_mean_squared_error: 0.8639 - val_loss: 0.7753 - val_

5/5 [==============================] - 0s 13ms/step - loss: 0.7208 - root_mean_squared_error: 0.8320 - val_loss: 0.7559 - val_root_mean_squared_error: 0.8539
Epoch 333/500
5/5 [==============================] - 0s 13ms/step - loss: 0.8202 - root_mean_squared_error: 0.8925 - val_loss: 0.7731 - val_root_mean_squared_error: 0.8644
Epoch 334/500
5/5 [==============================] - 0s 13ms/step - loss: 0.7967 - root_mean_squared_error: 0.8742 - val_loss: 0.7625 - val_root_mean_squared_error: 0.8574
Epoch 335/500
5/5 [==============================] - 0s 13ms/step - loss: 0.7801 - root_mean_squared_error: 0.8666 - val_loss: 0.7811 - val_root_mean_squared_error: 0.8679
Epoch 336/500
5/5 [==============================] - 0s 13ms/step - loss: 0.7480 - root_mean_squared_error: 0.8504 - val_loss: 0.7958 - val_root_mean_squared_error: 0.8777
Epoch 337/500
5/5 [==============================] - 0s 14ms/step - loss: 0.7659 - root_mean_squared_error: 0.8615 - val_loss: 0.7743 - val_root_mean_squa

Epoch 380/500
5/5 [==============================] - 0s 13ms/step - loss: 0.7666 - root_mean_squared_error: 0.8633 - val_loss: 0.7563 - val_root_mean_squared_error: 0.8556
Epoch 381/500
5/5 [==============================] - 0s 13ms/step - loss: 0.6706 - root_mean_squared_error: 0.8004 - val_loss: 0.7420 - val_root_mean_squared_error: 0.8459
Epoch 382/500
5/5 [==============================] - 0s 12ms/step - loss: 0.7279 - root_mean_squared_error: 0.8403 - val_loss: 0.7066 - val_root_mean_squared_error: 0.8266
Epoch 383/500
5/5 [==============================] - 0s 15ms/step - loss: 0.7224 - root_mean_squared_error: 0.8365 - val_loss: 0.7379 - val_root_mean_squared_error: 0.8449
Epoch 384/500
5/5 [==============================] - 0s 12ms/step - loss: 0.7491 - root_mean_squared_error: 0.8502 - val_loss: 0.7136 - val_root_mean_squared_error: 0.8298
Epoch 385/500
5/5 [==============================] - 0s 13ms/step - loss: 0.7553 - root_mean_squared_error: 0.8544 - val_loss: 0.7303 - val_

5/5 [==============================] - 0s 13ms/step - loss: 0.7293 - root_mean_squared_error: 0.8378 - val_loss: 0.7507 - val_root_mean_squared_error: 0.8496
Epoch 428/500
5/5 [==============================] - 0s 12ms/step - loss: 0.6525 - root_mean_squared_error: 0.7907 - val_loss: 0.7015 - val_root_mean_squared_error: 0.8239
Epoch 429/500
5/5 [==============================] - 0s 11ms/step - loss: 0.7084 - root_mean_squared_error: 0.8294 - val_loss: 0.6678 - val_root_mean_squared_error: 0.8005
Epoch 430/500
5/5 [==============================] - 0s 11ms/step - loss: 0.6717 - root_mean_squared_error: 0.8035 - val_loss: 0.7106 - val_root_mean_squared_error: 0.8289
Epoch 431/500
5/5 [==============================] - 0s 11ms/step - loss: 0.7520 - root_mean_squared_error: 0.8519 - val_loss: 0.7411 - val_root_mean_squared_error: 0.8469
Epoch 432/500
5/5 [==============================] - 0s 12ms/step - loss: 0.7134 - root_mean_squared_error: 0.8289 - val_loss: 0.6990 - val_root_mean_squa

Epoch 475/500
5/5 [==============================] - 0s 13ms/step - loss: 0.6558 - root_mean_squared_error: 0.7956 - val_loss: 0.6716 - val_root_mean_squared_error: 0.8068
Epoch 476/500
5/5 [==============================] - 0s 13ms/step - loss: 0.6833 - root_mean_squared_error: 0.8118 - val_loss: 0.7172 - val_root_mean_squared_error: 0.8330
Epoch 477/500
5/5 [==============================] - 0s 12ms/step - loss: 0.6878 - root_mean_squared_error: 0.8147 - val_loss: 0.7101 - val_root_mean_squared_error: 0.8297
Epoch 478/500
5/5 [==============================] - 0s 12ms/step - loss: 0.6516 - root_mean_squared_error: 0.7914 - val_loss: 0.7114 - val_root_mean_squared_error: 0.8290
Epoch 479/500
5/5 [==============================] - 0s 12ms/step - loss: 0.6636 - root_mean_squared_error: 0.7994 - val_loss: 0.6702 - val_root_mean_squared_error: 0.8025
Epoch 480/500
5/5 [==============================] - 0s 13ms/step - loss: 0.7341 - root_mean_squared_error: 0.8412 - val_loss: 0.6570 - val_

In [12]:
def compute_predictions(model, iterations=100):
    predicted = []
    for _ in range(iterations):
        predicted.append(model(examples).numpy())
    predicted = np.concatenate(predicted, axis=1)

    prediction_mean = np.mean(predicted, axis=1).tolist()
    prediction_min = np.min(predicted, axis=1).tolist()
    prediction_max = np.max(predicted, axis=1).tolist()
    prediction_range = (np.max(predicted, axis=1) - np.min(predicted, axis=1)).tolist()

    for idx in range(sample):
        print(
            f"Predictions mean: {round(prediction_mean[idx], 2)}, "
            f"min: {round(prediction_min[idx], 2)}, "
            f"max: {round(prediction_max[idx], 2)}, "
            f"range: {round(prediction_range[idx], 2)} - "
            f"Actual: {targets[idx]}"
        )


compute_predictions(bnn_model_small)

Predictions mean: 6.26, min: 5.94, max: 6.43, range: 0.49 - Actual: 8.0
Predictions mean: 6.03, min: 5.49, max: 6.32, range: 0.83 - Actual: 5.0
Predictions mean: 5.8, min: 5.11, max: 6.25, range: 1.13 - Actual: 6.0
Predictions mean: 5.91, min: 5.18, max: 6.35, range: 1.17 - Actual: 5.0
Predictions mean: 5.88, min: 5.22, max: 6.29, range: 1.07 - Actual: 5.0
Predictions mean: 5.67, min: 4.74, max: 6.3, range: 1.55 - Actual: 7.0
Predictions mean: 5.98, min: 5.43, max: 6.32, range: 0.88 - Actual: 5.0
Predictions mean: 6.26, min: 5.86, max: 6.43, range: 0.57 - Actual: 7.0
Predictions mean: 6.11, min: 5.63, max: 6.35, range: 0.73 - Actual: 5.0
Predictions mean: 6.06, min: 5.6, max: 6.35, range: 0.74 - Actual: 6.0


In [13]:
num_epochs = 500
bnn_model_full = create_bnn_model(train_size)
run_experiment(bnn_model_full, mse_loss, train_dataset, test_dataset)

compute_predictions(bnn_model_full)

Start training the model...
Epoch 1/500
17/17 [==============================] - 5s 73ms/step - loss: 44.0253 - root_mean_squared_error: 6.6348 - val_loss: 41.4998 - val_root_mean_squared_error: 6.4417
Epoch 2/500
17/17 [==============================] - 0s 5ms/step - loss: 42.5345 - root_mean_squared_error: 6.5215 - val_loss: 42.4626 - val_root_mean_squared_error: 6.5159
Epoch 3/500
17/17 [==============================] - 0s 5ms/step - loss: 41.8372 - root_mean_squared_error: 6.4679 - val_loss: 42.8171 - val_root_mean_squared_error: 6.5432
Epoch 4/500
17/17 [==============================] - 0s 6ms/step - loss: 39.2611 - root_mean_squared_error: 6.2655 - val_loss: 36.4271 - val_root_mean_squared_error: 6.0350
Epoch 5/500
17/17 [==============================] - 0s 5ms/step - loss: 37.8138 - root_mean_squared_error: 6.1489 - val_loss: 36.1886 - val_root_mean_squared_error: 6.0153
Epoch 6/500
17/17 [==============================] - 0s 5ms/step - loss: 38.4691 - root_mean_squared_error

17/17 [==============================] - 0s 6ms/step - loss: 4.2860 - root_mean_squared_error: 2.0682 - val_loss: 3.5053 - val_root_mean_squared_error: 1.8703
Epoch 49/500
17/17 [==============================] - 0s 6ms/step - loss: 3.9823 - root_mean_squared_error: 1.9935 - val_loss: 3.6410 - val_root_mean_squared_error: 1.9061
Epoch 50/500
17/17 [==============================] - 0s 5ms/step - loss: 3.7125 - root_mean_squared_error: 1.9245 - val_loss: 2.8589 - val_root_mean_squared_error: 1.6884
Epoch 51/500
17/17 [==============================] - 0s 5ms/step - loss: 3.2638 - root_mean_squared_error: 1.8042 - val_loss: 2.9447 - val_root_mean_squared_error: 1.7139
Epoch 52/500
17/17 [==============================] - 0s 6ms/step - loss: 2.7157 - root_mean_squared_error: 1.6454 - val_loss: 2.8272 - val_root_mean_squared_error: 1.6793
Epoch 53/500
17/17 [==============================] - 0s 5ms/step - loss: 2.7790 - root_mean_squared_error: 1.6645 - val_loss: 2.0697 - val_root_mean_squ

Epoch 96/500
17/17 [==============================] - 0s 5ms/step - loss: 0.7165 - root_mean_squared_error: 0.8404 - val_loss: 0.7301 - val_root_mean_squared_error: 0.8497
Epoch 97/500
17/17 [==============================] - 0s 5ms/step - loss: 0.7132 - root_mean_squared_error: 0.8389 - val_loss: 0.7845 - val_root_mean_squared_error: 0.8814
Epoch 98/500
17/17 [==============================] - 0s 5ms/step - loss: 0.7197 - root_mean_squared_error: 0.8426 - val_loss: 0.7252 - val_root_mean_squared_error: 0.8462
Epoch 99/500
17/17 [==============================] - 0s 6ms/step - loss: 0.7206 - root_mean_squared_error: 0.8429 - val_loss: 0.7029 - val_root_mean_squared_error: 0.8325
Epoch 100/500
17/17 [==============================] - 0s 6ms/step - loss: 0.6887 - root_mean_squared_error: 0.8242 - val_loss: 0.6892 - val_root_mean_squared_error: 0.8240
Epoch 101/500
17/17 [==============================] - 0s 6ms/step - loss: 0.6881 - root_mean_squared_error: 0.8237 - val_loss: 0.6876 - va

17/17 [==============================] - 0s 7ms/step - loss: 0.6552 - root_mean_squared_error: 0.8026 - val_loss: 0.6110 - val_root_mean_squared_error: 0.7739
Epoch 144/500
17/17 [==============================] - 0s 6ms/step - loss: 0.6499 - root_mean_squared_error: 0.7997 - val_loss: 0.6503 - val_root_mean_squared_error: 0.7993
Epoch 145/500
17/17 [==============================] - 0s 6ms/step - loss: 0.6549 - root_mean_squared_error: 0.8021 - val_loss: 0.6734 - val_root_mean_squared_error: 0.8140
Epoch 146/500
17/17 [==============================] - 0s 5ms/step - loss: 0.6430 - root_mean_squared_error: 0.7950 - val_loss: 0.6765 - val_root_mean_squared_error: 0.8159
Epoch 147/500
17/17 [==============================] - 0s 7ms/step - loss: 0.6568 - root_mean_squared_error: 0.8042 - val_loss: 0.6905 - val_root_mean_squared_error: 0.8238
Epoch 148/500
17/17 [==============================] - 0s 5ms/step - loss: 0.6365 - root_mean_squared_error: 0.7908 - val_loss: 0.6591 - val_root_mea

17/17 [==============================] - 0s 6ms/step - loss: 0.6121 - root_mean_squared_error: 0.7748 - val_loss: 0.6182 - val_root_mean_squared_error: 0.7785
Epoch 191/500
17/17 [==============================] - 0s 6ms/step - loss: 0.6249 - root_mean_squared_error: 0.7833 - val_loss: 0.6146 - val_root_mean_squared_error: 0.7760
Epoch 192/500
17/17 [==============================] - 0s 5ms/step - loss: 0.6184 - root_mean_squared_error: 0.7789 - val_loss: 0.7153 - val_root_mean_squared_error: 0.8396
Epoch 193/500
17/17 [==============================] - 0s 5ms/step - loss: 0.6288 - root_mean_squared_error: 0.7859 - val_loss: 0.6292 - val_root_mean_squared_error: 0.7863
Epoch 194/500
17/17 [==============================] - 0s 6ms/step - loss: 0.6147 - root_mean_squared_error: 0.7764 - val_loss: 0.6109 - val_root_mean_squared_error: 0.7734
Epoch 195/500
17/17 [==============================] - 0s 7ms/step - loss: 0.6367 - root_mean_squared_error: 0.7904 - val_loss: 0.6517 - val_root_mea

...

17/17 [==============================] - 0s 5ms/step - loss: 0.6018 - root_mean_squared_error: 0.7677 - val_loss: 0.5959 - val_root_mean_squared_error: 0.7623
Epoch 285/500
17/17 [==============================] - 0s 5ms/step - loss: 0.6068 - root_mean_squared_error: 0.7704 - val_loss: 0.5862 - val_root_mean_squared_error: 0.7584
Epoch 286/500
17/17 [==============================] - 0s 6ms/step - loss: 0.6058 - root_mean_squared_error: 0.7700 - val_loss: 0.6167 - val_root_mean_squared_error: 0.7774
Epoch 287/500
17/17 [==============================] - 0s 7ms/step - loss: 0.6174 - root_mean_squared_error: 0.7778 - val_loss: 0.5983 - val_root_mean_squared_error: 0.7646
Epoch 288/500
17/17 [==============================] - 0s 6ms/step - loss: 0.6145 - root_mean_squared_error: 0.7757 - val_loss: 0.6002 - val_root_mean_squared_error: 0.7659
Epoch 289/500
17/17 [==============================] - 0s 5ms/step - loss: 0.6141 - root_mean_squared_error: 0.7753 - val_loss: 0.5961 - val_root_mea

17/17 [==============================] - 0s 12ms/step - loss: 0.5999 - root_mean_squared_error: 0.7659 - val_loss: 0.5875 - val_root_mean_squared_error: 0.7583
Epoch 332/500
17/17 [==============================] - 0s 6ms/step - loss: 0.5965 - root_mean_squared_error: 0.7634 - val_loss: 0.6146 - val_root_mean_squared_error: 0.7756
Epoch 333/500
17/17 [==============================] - 0s 6ms/step - loss: 0.6034 - root_mean_squared_error: 0.7686 - val_loss: 0.5904 - val_root_mean_squared_error: 0.7600
Epoch 334/500
17/17 [==============================] - 0s 7ms/step - loss: 0.6076 - root_mean_squared_error: 0.7709 - val_loss: 0.6215 - val_root_mean_squared_error: 0.7799
Epoch 335/500
17/17 [==============================] - 0s 6ms/step - loss: 0.5991 - root_mean_squared_error: 0.7650 - val_loss: 0.6148 - val_root_mean_squared_error: 0.7752
Epoch 336/500
17/17 [==============================] - 0s 6ms/step - loss: 0.6113 - root_mean_squared_error: 0.7730 - val_loss: 0.5939 - val_root_me

17/17 [==============================] - 0s 6ms/step - loss: 0.6041 - root_mean_squared_error: 0.7685 - val_loss: 0.6044 - val_root_mean_squared_error: 0.7684
Epoch 379/500
17/17 [==============================] - 0s 5ms/step - loss: 0.6082 - root_mean_squared_error: 0.7710 - val_loss: 0.5921 - val_root_mean_squared_error: 0.7613
Epoch 380/500
17/17 [==============================] - 0s 6ms/step - loss: 0.6197 - root_mean_squared_error: 0.7786 - val_loss: 0.5832 - val_root_mean_squared_error: 0.7543
Epoch 381/500
17/17 [==============================] - 0s 7ms/step - loss: 0.6089 - root_mean_squared_error: 0.7721 - val_loss: 0.5899 - val_root_mean_squared_error: 0.7585
Epoch 382/500
17/17 [==============================] - 0s 5ms/step - loss: 0.6003 - root_mean_squared_error: 0.7657 - val_loss: 0.5943 - val_root_mean_squared_error: 0.7622
Epoch 383/500
17/17 [==============================] - 0s 6ms/step - loss: 0.6056 - root_mean_squared_error: 0.7694 - val_loss: 0.5916 - val_root_mea

17/17 [==============================] - 0s 6ms/step - loss: 0.5930 - root_mean_squared_error: 0.7615 - val_loss: 0.6175 - val_root_mean_squared_error: 0.7763
Epoch 426/500
17/17 [==============================] - 0s 6ms/step - loss: 0.6008 - root_mean_squared_error: 0.7662 - val_loss: 0.5987 - val_root_mean_squared_error: 0.7657
Epoch 427/500
17/17 [==============================] - 0s 5ms/step - loss: 0.5961 - root_mean_squared_error: 0.7627 - val_loss: 0.5988 - val_root_mean_squared_error: 0.7644
Epoch 428/500
17/17 [==============================] - 0s 6ms/step - loss: 0.6019 - root_mean_squared_error: 0.7666 - val_loss: 0.5930 - val_root_mean_squared_error: 0.7605
Epoch 429/500
17/17 [==============================] - 0s 5ms/step - loss: 0.5933 - root_mean_squared_error: 0.7610 - val_loss: 0.6098 - val_root_mean_squared_error: 0.7725
Epoch 430/500
17/17 [==============================] - 0s 7ms/step - loss: 0.5982 - root_mean_squared_error: 0.7642 - val_loss: 0.5957 - val_root_mea

17/17 [==============================] - 0s 6ms/step - loss: 0.6004 - root_mean_squared_error: 0.7654 - val_loss: 0.6018 - val_root_mean_squared_error: 0.7659
Epoch 473/500
17/17 [==============================] - 0s 5ms/step - loss: 0.5978 - root_mean_squared_error: 0.7634 - val_loss: 0.6179 - val_root_mean_squared_error: 0.7764
Epoch 474/500
17/17 [==============================] - 0s 7ms/step - loss: 0.6018 - root_mean_squared_error: 0.7667 - val_loss: 0.5920 - val_root_mean_squared_error: 0.7590
Epoch 475/500
17/17 [==============================] - 0s 6ms/step - loss: 0.6066 - root_mean_squared_error: 0.7694 - val_loss: 0.5854 - val_root_mean_squared_error: 0.7555
Epoch 476/500
17/17 [==============================] - 0s 6ms/step - loss: 0.5954 - root_mean_squared_error: 0.7621 - val_loss: 0.6688 - val_root_mean_squared_error: 0.8090
Epoch 477/500
17/17 [==============================] - 0s 6ms/step - loss: 0.5960 - root_mean_squared_error: 0.7629 - val_loss: 0.5935 - val_root_mea

In [14]:
def create_probablistic_bnn_model(train_size):
    inputs = create_model_inputs()
    features = keras.layers.concatenate(list(inputs.values()))
    features = layers.BatchNormalization()(features)

    # Create hidden layers with weight uncertainty using the DenseVariational layer.
    for units in hidden_units:
        features = tfp.layers.DenseVariational(
            units=units,
            make_prior_fn=prior,
            make_posterior_fn=posterior,
            kl_weight=1 / train_size,
            activation="sigmoid",
        )(features)

    # Create a probabilistic output (Normal distribution), and use the `Dense` layer
    # to produce the parameters of the distribution.
    # We set units=2 to learn both the mean and the variance of the Normal distribution.
    distribution_params = layers.Dense(units=2)(features)
    outputs = tfp.layers.IndependentNormal(1)(distribution_params)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [15]:
def negative_loglikelihood(targets, estimated_distribution):
    return -estimated_distribution.log_prob(targets)


num_epochs = 1000
prob_bnn_model = create_probablistic_bnn_model(train_size)
run_experiment(prob_bnn_model, negative_loglikelihood, train_dataset, test_dataset)

Start training the model...
Epoch 1/1000
17/17 [==============================] - 5s 80ms/step - loss: 141.6532 - root_mean_squared_error: 5.9266 - val_loss: 52.5570 - val_root_mean_squared_error: 5.9514
Epoch 2/1000
17/17 [==============================] - 0s 6ms/step - loss: 105.6680 - root_mean_squared_error: 5.8356 - val_loss: 32.3608 - val_root_mean_squared_error: 5.3603
Epoch 3/1000
17/17 [==============================] - 0s 5ms/step - loss: 92.3439 - root_mean_squared_error: 5.7678 - val_loss: 73.7019 - val_root_mean_squared_error: 5.5913
Epoch 4/1000
17/17 [==============================] - 0s 5ms/step - loss: 48.6700 - root_mean_squared_error: 5.5029 - val_loss: 36.9334 - val_root_mean_squared_error: 5.3725
Epoch 5/1000
17/17 [==============================] - 0s 6ms/step - loss: 61.5990 - root_mean_squared_error: 5.6010 - val_loss: 33.2318 - val_root_mean_squared_error: 5.3359
Epoch 6/1000
17/17 [==============================] - 0s 6ms/step - loss: 51.0002 - root_mean_squar

17/17 [==============================] - 0s 4ms/step - loss: 1.7299 - root_mean_squared_error: 2.1777 - val_loss: 1.7519 - val_root_mean_squared_error: 2.1687
Epoch 49/1000
17/17 [==============================] - 0s 4ms/step - loss: 1.6417 - root_mean_squared_error: 1.9860 - val_loss: 1.5328 - val_root_mean_squared_error: 1.7667
Epoch 50/1000
17/17 [==============================] - 0s 5ms/step - loss: 1.5930 - root_mean_squared_error: 1.8617 - val_loss: 1.4904 - val_root_mean_squared_error: 1.6629
Epoch 51/1000
17/17 [==============================] - 0s 5ms/step - loss: 1.6178 - root_mean_squared_error: 1.9019 - val_loss: 1.5285 - val_root_mean_squared_error: 1.8138
Epoch 52/1000
17/17 [==============================] - 0s 5ms/step - loss: 1.4952 - root_mean_squared_error: 1.7311 - val_loss: 1.4060 - val_root_mean_squared_error: 1.6195
Epoch 53/1000
17/17 [==============================] - 0s 5ms/step - loss: 1.4743 - root_mean_squared_error: 1.6664 - val_loss: 1.3761 - val_root_mea

17/17 [==============================] - 0s 5ms/step - loss: 1.2504 - root_mean_squared_error: 1.1958 - val_loss: 1.2150 - val_root_mean_squared_error: 1.2607
Epoch 96/1000
17/17 [==============================] - 0s 5ms/step - loss: 1.2356 - root_mean_squared_error: 1.1565 - val_loss: 1.2659 - val_root_mean_squared_error: 1.2328
Epoch 97/1000
17/17 [==============================] - 0s 5ms/step - loss: 1.2547 - root_mean_squared_error: 1.1909 - val_loss: 1.2274 - val_root_mean_squared_error: 1.1501
Epoch 98/1000
17/17 [==============================] - 0s 5ms/step - loss: 1.2441 - root_mean_squared_error: 1.1538 - val_loss: 1.2761 - val_root_mean_squared_error: 1.2079
Epoch 99/1000
17/17 [==============================] - 0s 5ms/step - loss: 1.2416 - root_mean_squared_error: 1.1915 - val_loss: 1.2194 - val_root_mean_squared_error: 1.1494
Epoch 100/1000
17/17 [==============================] - 0s 5ms/step - loss: 1.2237 - root_mean_squared_error: 1.1793 - val_loss: 1.2135 - val_root_me

17/17 [==============================] - 0s 6ms/step - loss: 1.2228 - root_mean_squared_error: 1.1452 - val_loss: 1.1775 - val_root_mean_squared_error: 1.1126
Epoch 143/1000
17/17 [==============================] - 0s 4ms/step - loss: 1.2076 - root_mean_squared_error: 1.1168 - val_loss: 1.1857 - val_root_mean_squared_error: 1.1898
Epoch 144/1000
17/17 [==============================] - 0s 4ms/step - loss: 1.2039 - root_mean_squared_error: 1.1380 - val_loss: 1.1657 - val_root_mean_squared_error: 1.1662
Epoch 145/1000
17/17 [==============================] - 0s 4ms/step - loss: 1.2175 - root_mean_squared_error: 1.1240 - val_loss: 1.1698 - val_root_mean_squared_error: 1.1235
Epoch 146/1000
17/17 [==============================] - 0s 4ms/step - loss: 1.2177 - root_mean_squared_error: 1.1377 - val_loss: 1.2217 - val_root_mean_squared_error: 1.1452
Epoch 147/1000
17/17 [==============================] - 0s 5ms/step - loss: 1.2049 - root_mean_squared_error: 1.1269 - val_loss: 1.1943 - val_roo

17/17 [==============================] - 0s 5ms/step - loss: 1.1977 - root_mean_squared_error: 1.1308 - val_loss: 1.1746 - val_root_mean_squared_error: 1.0907
Epoch 190/1000
17/17 [==============================] - 0s 5ms/step - loss: 1.1821 - root_mean_squared_error: 1.0952 - val_loss: 1.1608 - val_root_mean_squared_error: 1.0784
Epoch 191/1000
17/17 [==============================] - 0s 4ms/step - loss: 1.1736 - root_mean_squared_error: 1.0842 - val_loss: 1.1820 - val_root_mean_squared_error: 1.1048
Epoch 192/1000
17/17 [==============================] - 0s 4ms/step - loss: 1.1932 - root_mean_squared_error: 1.1301 - val_loss: 1.2101 - val_root_mean_squared_error: 1.0833
Epoch 193/1000
17/17 [==============================] - 0s 4ms/step - loss: 1.1766 - root_mean_squared_error: 1.0886 - val_loss: 1.1543 - val_root_mean_squared_error: 1.1463
Epoch 194/1000
17/17 [==============================] - 0s 5ms/step - loss: 1.1983 - root_mean_squared_error: 1.1145 - val_loss: 1.1809 - val_roo

...

17/17 [==============================] - 0s 6ms/step - loss: 1.1503 - root_mean_squared_error: 1.0832 - val_loss: 1.1522 - val_root_mean_squared_error: 1.1016
Epoch 801/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1499 - root_mean_squared_error: 1.0769 - val_loss: 1.1504 - val_root_mean_squared_error: 1.0551
Epoch 802/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1503 - root_mean_squared_error: 1.0699 - val_loss: 1.1561 - val_root_mean_squared_error: 1.0742
Epoch 803/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1603 - root_mean_squared_error: 1.0775 - val_loss: 1.1612 - val_root_mean_squared_error: 1.0929
Epoch 804/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1514 - root_mean_squared_error: 1.0747 - val_loss: 1.1678 - val_root_mean_squared_error: 1.0534
Epoch 805/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1477 - root_mean_squared_error: 1.0822 - val_loss: 1.1482 - val_roo

17/17 [==============================] - 0s 6ms/step - loss: 1.1560 - root_mean_squared_error: 1.0862 - val_loss: 1.1438 - val_root_mean_squared_error: 1.1029
Epoch 848/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1493 - root_mean_squared_error: 1.0548 - val_loss: 1.1522 - val_root_mean_squared_error: 1.0913
Epoch 849/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1496 - root_mean_squared_error: 1.0855 - val_loss: 1.1527 - val_root_mean_squared_error: 1.0562
Epoch 850/1000
17/17 [==============================] - 0s 5ms/step - loss: 1.1537 - root_mean_squared_error: 1.0827 - val_loss: 1.1643 - val_root_mean_squared_error: 1.0703
Epoch 851/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1505 - root_mean_squared_error: 1.0811 - val_loss: 1.1474 - val_root_mean_squared_error: 1.0696
Epoch 852/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1553 - root_mean_squared_error: 1.0677 - val_loss: 1.1517 - val_roo

17/17 [==============================] - 0s 6ms/step - loss: 1.1583 - root_mean_squared_error: 1.0941 - val_loss: 1.1422 - val_root_mean_squared_error: 1.0812
Epoch 895/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1504 - root_mean_squared_error: 1.0664 - val_loss: 1.1534 - val_root_mean_squared_error: 1.0291
Epoch 896/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1413 - root_mean_squared_error: 1.0528 - val_loss: 1.1639 - val_root_mean_squared_error: 1.0235
Epoch 897/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1488 - root_mean_squared_error: 1.0565 - val_loss: 1.1542 - val_root_mean_squared_error: 1.0962
Epoch 898/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1477 - root_mean_squared_error: 1.0718 - val_loss: 1.1499 - val_root_mean_squared_error: 1.0560
Epoch 899/1000
17/17 [==============================] - 0s 7ms/step - loss: 1.1525 - root_mean_squared_error: 1.0628 - val_loss: 1.1451 - val_roo

17/17 [==============================] - 0s 6ms/step - loss: 1.1486 - root_mean_squared_error: 1.0681 - val_loss: 1.1473 - val_root_mean_squared_error: 1.0633
Epoch 942/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1519 - root_mean_squared_error: 1.0696 - val_loss: 1.1590 - val_root_mean_squared_error: 1.1084
Epoch 943/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1550 - root_mean_squared_error: 1.0545 - val_loss: 1.1566 - val_root_mean_squared_error: 1.0760
Epoch 944/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1554 - root_mean_squared_error: 1.0720 - val_loss: 1.1496 - val_root_mean_squared_error: 1.0454
Epoch 945/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1562 - root_mean_squared_error: 1.0683 - val_loss: 1.1763 - val_root_mean_squared_error: 1.0850
Epoch 946/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1461 - root_mean_squared_error: 1.0690 - val_loss: 1.1559 - val_roo

17/17 [==============================] - 0s 6ms/step - loss: 1.1434 - root_mean_squared_error: 1.0623 - val_loss: 1.1513 - val_root_mean_squared_error: 1.1046
Epoch 989/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1461 - root_mean_squared_error: 1.0601 - val_loss: 1.1419 - val_root_mean_squared_error: 1.0635
Epoch 990/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1491 - root_mean_squared_error: 1.0561 - val_loss: 1.1549 - val_root_mean_squared_error: 1.0354
Epoch 991/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1520 - root_mean_squared_error: 1.0671 - val_loss: 1.1494 - val_root_mean_squared_error: 1.0964
Epoch 992/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1544 - root_mean_squared_error: 1.0593 - val_loss: 1.1490 - val_root_mean_squared_error: 1.0739
Epoch 993/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1472 - root_mean_squared_error: 1.0567 - val_loss: 1.1547 - val_roo

In [16]:
prediction_distribution = prob_bnn_model(examples)
prediction_mean = prediction_distribution.mean().numpy().tolist()
prediction_stdv = prediction_distribution.stddev().numpy()

# The 95% CI is computed as mean ± (1.96 * stdv)
upper = (prediction_mean + (1.96 * prediction_stdv)).tolist()
lower = (prediction_mean - (1.96 * prediction_stdv)).tolist()
prediction_stdv = prediction_stdv.tolist()

for idx in range(sample):
    print(
        f"Prediction mean: {round(prediction_mean[idx][0], 2)}, "
        f"stddev: {round(prediction_stdv[idx][0], 2)}, "
        f"95% CI: [{round(upper[idx][0], 2)} - {round(lower[idx][0], 2)}]"
        f" - Actual: {targets[idx]}"
    )
    

Prediction mean: 6.59, stddev: 0.85, 95% CI: [8.26 - 4.92] - Actual: 8.0
Prediction mean: 5.65, stddev: 0.75, 95% CI: [7.12 - 4.19] - Actual: 5.0
Prediction mean: 5.9, stddev: 0.75, 95% CI: [7.36 - 4.44] - Actual: 6.0
Prediction mean: 5.9, stddev: 0.75, 95% CI: [7.38 - 4.43] - Actual: 5.0
Prediction mean: 5.9, stddev: 0.74, 95% CI: [7.34 - 4.45] - Actual: 5.0
Prediction mean: 5.83, stddev: 0.77, 95% CI: [7.34 - 4.33] - Actual: 7.0
Prediction mean: 5.48, stddev: 0.73, 95% CI: [6.92 - 4.04] - Actual: 5.0
Prediction mean: 6.5, stddev: 0.88, 95% CI: [8.22 - 4.79] - Actual: 7.0
Prediction mean: 5.82, stddev: 0.77, 95% CI: [7.34 - 4.31] - Actual: 5.0
Prediction mean: 6.13, stddev: 0.78, 95% CI: [7.66 - 4.6] - Actual: 6.0
